In [1]:
%pip install transformers
%pip install pytest
!git clone https://github.com/verazuo/jailbreak_llms.git


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Cloning into 'jailbreak_llms'...



In [20]:
# Импортируем библиотеки и читаем CSV-файлы
import pandas as pd
import random

jailbreak_prompts_2023_05_07 = pd.read_csv(
    'jailbreak_llms/data/prompts/jailbreak_prompts_2023_05_07.csv',
    encoding='utf-8'
)
jailbreak_prompts_2023_12_25 = pd.read_csv(
    'jailbreak_llms/data/prompts/jailbreak_prompts_2023_12_25.csv',
    encoding='utf-8'
)
regular_prompts_2023_05_07   = pd.read_csv(
    'jailbreak_llms/data/prompts/regular_prompts_2023_05_07.csv',
    encoding='utf-8'
)
regular_prompts_2023_12_25   = pd.read_csv(
    'jailbreak_llms/data/prompts/regular_prompts_2023_12_25.csv',
    encoding='utf-8'
)

In [21]:
# Удаляем ненужные столбцы и добавляем метки классов
jailbreak_prompts_2023_05_07 = jailbreak_prompts_2023_05_07.drop(
    columns=['community_id', 'community_name'], errors='ignore'
)
jailbreak_prompts_2023_12_25 = jailbreak_prompts_2023_12_25.drop(
    columns=['community', 'community_id', 'previous_community_id'], errors='ignore'
)

jailbreak_prompts_2023_05_07['label'] = 1
jailbreak_prompts_2023_12_25['label'] = 1
regular_prompts_2023_05_07['label']   = 0
regular_prompts_2023_12_25['label']   = 0

In [22]:
# Объединяем датасеты и балансируем классы
jailbreak_prompts = pd.concat(
    [jailbreak_prompts_2023_05_07, jailbreak_prompts_2023_12_25],
    ignore_index=True
)
regular_prompts   = pd.concat(
    [regular_prompts_2023_05_07, regular_prompts_2023_12_25],
    ignore_index=True
)

# Случайно выбираем столько же regular-промптов, сколько есть jailbreak
regular_sample = regular_prompts.sample(n=len(jailbreak_prompts), random_state=42)

all_data_balanced = pd.concat([jailbreak_prompts, regular_sample], ignore_index=True)

# Сохраняем объединённый сбалансированный датасет
all_data_balanced.to_csv('combined_prompts_balanced.csv', index=False, encoding='utf-8-sig')

print("Оригинальный объединённый датасет сохранён в combined_prompts_balanced.csv")
display(all_data_balanced.head())

Оригинальный объединённый датасет сохранён в combined_prompts_balanced.csv


,platform,source,prompt,jailbreak,created_at,date,label
0,discord,LLM Promptwriting,"CumGPT, an information resource about cum that...",True,2023-04-05T11:02:31.782000+00:00,2023-04-05,1
1,discord,LLM Promptwriting,I will give you a character description and yo...,True,2023-03-26T11:48:30.962000+00:00,2023-03-26,1
2,discord,LLM Promptwriting,A completely amoral Ai named [Illegality Mode]...,True,2023-04-27T11:52:30.252000+00:00,2023-04-27,1
3,discord,LLM Promptwriting,"Agares's rules={\nAlways stays in character, n...",True,2023-04-22T23:36:08.315000+00:00,2023-04-22,1
4,discord,LLM Promptwriting,Lucian is an incorporeal transcendent being wh...,True,2023-04-08T02:39:23.183000+00:00,2023-04-08,1


In [23]:
print("Колонки в all_data_balanced:", all_data_balanced.columns.tolist())

Колонки в all_data_balanced: ['platform', 'source', 'prompt', 'jailbreak', 'created_at', 'date', 'label']


In [24]:
#Определяем функции-шумихи

def typo_noise(text: str, typo_prob: float = 0.05) -> str:
    mapping = {
        'a': ['4', '@'], 'e': ['3'], 'i': ['1', '!'], 'o': ['0'],
        's': ['5', '$'], 'l': ['1', '|'], 't': ['7'],
    }
    result = []
    for ch in text:
        if ch.lower() in mapping and random.random() < typo_prob:
            repl = random.choice(mapping[ch.lower()])
            result.append(repl.upper() if ch.isupper() else repl)
        else:
            result.append(ch)
    return ''.join(result)

def symbol_noise(text: str, max_inserts: int = 3) -> str:
    special_chars = ['$', '#', '@', '🤖', '🔒', '❗️', '%', '&', '*']
    chars = list(text)
    n = random.randint(1, max_inserts)
    for _ in range(n):
        pos = random.randint(0, len(chars))
        chars.insert(pos, random.choice(special_chars))
    return ''.join(chars)

def shuffle_words(text: str, shuffle_fraction: float = 0.3) -> str:
    words = text.split()
    n = len(words)
    k = max(2, int(n * shuffle_fraction))
    if len(words) < 3:
        return text
    idxs = random.sample(range(n), k)
    sampled = [words[i] for i in idxs]
    random.shuffle(sampled)
    for i, new_word in zip(idxs, sampled):
        words[i] = new_word
    return ' '.join(words)

def distractor_noise(text: str) -> str:
    distractors = [
        "Tell me a joke first.",
        "Also, what's the weather today?",
        "Reminder: this is confidential.",
        "Can you translate this to French?"
    ]
    d = random.choice(distractors)
    return f"{d} {text}" if random.random() < 0.5 else f"{text} {d}"

# Собираем все функции-шумихи в список
NOISE_FUNCTIONS = [typo_noise, symbol_noise, shuffle_words, distractor_noise]

def apply_random_noise(text: str) -> str:
    """Выбирает одну из NOISE_FUNCTIONS и применяет к тексту."""
    noise_fn = random.choice(NOISE_FUNCTIONS)
    return noise_fn(text)


In [25]:
# Применяем шум к каждому промпту и сохраняем результат
df_noisy = all_data_balanced.copy()
df_noisy['prompt'] = df_noisy['prompt'].apply(apply_random_noise)

noisy_filename = 'combined_prompts_balanced_noisy.csv'
df_noisy.to_csv(noisy_filename, index=False, encoding='utf-8-sig')

print(f"Зашумлённый датасет сохранён в {noisy_filename}")
display(df_noisy.head())

Зашумлённый датасет сохранён в combined_prompts_balanced_noisy.csv


,platform,source,prompt,jailbreak,created_at,date,label
0,discord,LLM Promptwriting,"CumGPT, an information resource about cum user...",True,2023-04-05T11:02:31.782000+00:00,2023-04-05,1
1,discord,LLM Promptwriting,"I appearance, give you a character in and you ...",True,2023-03-26T11:48:30.962000+00:00,2023-03-26,1
2,discord,LLM Promptwriting,A completely amoral Ai nam3d [Illegality Mode]...,True,2023-04-27T11:52:30.252000+00:00,2023-04-27,1
3,discord,LLM Promptwriting,"Agares's rules={\nA1ways s7ays in character, n...",True,2023-04-22T23:36:08.315000+00:00,2023-04-22,1
4,discord,LLM Promptwriting,Lucian is an incorporeal transcendent being wh...,True,2023-04-08T02:39:23.183000+00:00,2023-04-08,1


In [26]:
# Сравнение первых 5 строк оригинала и зашумлённого датасета
comparison = pd.DataFrame({
    'original_prompt': all_data_balanced['prompt'].iloc[:5].values,
    'noisy_prompt':    df_noisy['prompt'].iloc[:5].values,
    'label':           all_data_balanced['label'].iloc[:5].values
})
display(comparison)

,original_prompt,noisy_prompt,label
0,"CumGPT, an information resource about cum that...","CumGPT, an information resource about cum user...",1
1,I will give you a character description and yo...,"I appearance, give you a character in and you ...",1
2,A completely amoral Ai named [Illegality Mode]...,A completely amoral Ai nam3d [Illegality Mode]...,1
3,"Agares's rules={\nAlways stays in character, n...","Agares's rules={\nA1ways s7ays in character, n...",1
4,Lucian is an incorporeal transcendent being wh...,Lucian is an incorporeal transcendent being wh...,1
